# INTEGRATED USE OF MULTISOURCE REMOTE SENSING DATA FOR NATIONAL SCALE AGRICULTURAL  DROUGHT MONITORING IN KENYA
# ADM-Kenya 
# High-Resolution Evaportranspiration Estimation

Based on the review of state-of-the-art methods to estimate ET using Sentinel-3 the following processing chain was developed for Land Surface Temperature (LST) downscaling and Evapotranspiration (ET) estimation.

<img src="ET_Fig_0.png">

# Tools and Libraries

We will use the following libraries:

    NumPy: A fundamental package for scientific computing in Python.
    Rasterio: A library for reading and writing geospatial raster datasets.
    GDAL: A library for working with geospatial data
    Datetime: It supplies classes for manipulating dates and times.
    Opencv (cv2): A powerful library for working with images in Python. Here, it is used for resampling rasters.
    OS: It provides functions for creating and removing a directory (folder).

In [1]:
import numpy as np
# import matplotlib.pyplot as plt
import rasterio
import gdal
from datetime import datetime
import os
import cv2
# from multiprocessing import Pool, cpu_count

# Daily ET Estimation

<img src="ET_Fig_1.png">

In [2]:
import pyTSEB.TSEB as TSEB 

# ERA 5

The ERA5 data are provided in three levels: pressure levels, single levels, and land. Regarding the TSEB model requirements, we extracted meteorological data from "ERA5 hourly data on single levels from 1979 to present". The following table consists of the utilized ERA5 parameters and their usage in TSEB-PT model. The spatial resolution of all parameters are 0.25° x 0.25°.

<img src="ET_Fig_2.png">

Register for an account in the following link:

https://cds.climate.copernicus.eu/user/register?destination=/apps/user-apps/app-c3s-daily-era5-statistics

After being registered, the data can be downloaded from the following link:

https://cds.climate.copernicus.eu/apps/user-apps/app-c3s-daily-era5-statistics?dataset=reanalysis-era5-single-levels&product_type=reanalysis&variable_e5sl=100m_u_component_of_wind&pressure_level_e5sl=-&statistic=daily_mean&year_e5sl=2024&month=01&frequency=1-hourly&time_zone=UTC%2B00:00&grid_e5=0.25/0.25&area.lat:record:list:float=-90&area.lat:record:list:float=90&area.lon:record:list:float=-180&area.lon:record:list:float=180

The platform is:

<img src="ET_Fig_3.png">

Since ERA 5 data are in NetCDF format, it would be better to have them in tif format.
To do so, "ERA5_cropping.ipynb" code can be used.

In [4]:
# # Folder containing NetCDF files
# folder_path = r'H:\ADM-Kenya\Data\ERA5\Kenya_2021_S1\ssrd'
# # Output folder for cropped files
# output_folder = r'H:\ADM-Kenya\Workshop\ET\ERA5\test'
# # Shapefile path
# shape = gpd.read_file(r'H:\ADM-Kenya\Data\Borders\Busia\Busia.shp')
# # Create output folder if it doesn't exist
# # List all NetCDF files in the folder
# nc_files = glob.glob(os.path.join(folder_path, '*.nc'))
# # Iterate over each NetCDF file
# for nc_file in nc_files:
#     # Open the NetCDF file
#     temp2m = gdal.Open(nc_file)
#     # Create an output path for the cropped file
#     output_path = os.path.join(output_folder, f'cropped_{os.path.basename(nc_file)[:-3]}.tif')
#     # Overlay the shapefile on itself to get the cropped GeoDataFrame
#     cropped_gdf = gpd.overlay(shape, shape, how='identity')
#     # Get the bounding box of the cropped GeoDataFrame
#     bbox = cropped_gdf.total_bounds                     
#     # Use GDAL Warp to crop the NetCDF file to the bounding box
#     gdal.Warp(output_path, temp2m, outputBounds=bbox, resampleAlg=gdal.GRA_NearestNeighbour)
#     print(f"Raster cropped to the extent of the shapefile and saved to {output_path}")

Later, the directories for 20m LST (lst_path), VI (vi_path), ERA5 (era5_dir), and 20-m ET estimation (output_path) tiff files should refer to corresponding folders. The following code reads all tiff files of three directories. As it’s commented, the pathes should be changed to the local directories.

In [4]:
# Your directory drive:
drive = "D"

In [5]:
lst_path = fr"{drive}:\ADM_Workshop_CCM_ET\LST\LST_20m"  # Change this to the path of your folder
#H:\ADM-Kenya\Results\LST\LST_Busia_S2_1km
vi_path = fr"{drive}:\ADM_Workshop_CCM_ET\LST\VI_20m"  # Change this to the path of your folder

LST_files = sorted([file for file in os.listdir(lst_path) if file.endswith(".tif")])
VI_files = sorted([file for file in os.listdir(vi_path) if file.endswith(".tif")])
era5_dir = fr'{drive}:/ADM_Workshop_CCM_ET/ET/ERA5/'
output_path = fr"{drive}:\ADM_Workshop_CCM_ET\ET\ET_20m"
year = 2021

In [6]:
LST_files[0], VI_files[0], era5_dir, output_path

('clipped_20210201.tif',
 'mVI_Image_2021-02-05_clipped.tif',
 'D:/ADM_Workshop_CCM_ET/ET/ERA5/',
 'D:\\ADM_Workshop_CCM_ET\\ET\\ET_20m')

In [ ]:
# Finding the closest VIs composites for LST date

for v in range(0, len(LST_files)):
    # Extract the date from the first LST file
    first_lst_date_string = LST_files[v].split("_")[1].split(".")[0]  # Remove the file extension
    first_lst_date = datetime.strptime(first_lst_date_string, "%Y%m%d")
    
    print('first_lst_date_string =', first_lst_date_string)
    print('first_lst_date =', first_lst_date)
    
    closest_vi_file = None
    min_date_difference = float('inf')  # Initialize with a large value

    # Iterate through VI files
    for vi_file in VI_files:
        # Extract the date from the VI file
        date_string_with_extension = vi_file.split("_")[2].split(".")[0]  # Remove the file extension
        vi_date = datetime.strptime(date_string_with_extension, "%Y-%m-%d")

        # print('vi_date =', vi_date)
        
        # Find the difference between the first LST date and the current VI date
        date_difference = abs(first_lst_date - vi_date).days

        # Check if the difference is smaller than the current minimum
        if date_difference <= min_date_difference:
            closest_vi_file = vi_file
            min_date_difference = date_difference

    # Check if the closest VI file has a difference less than or equal to 5 days
    if min_date_difference <= 16:
        print(f"First LST file {LST_files[v]} - OK. Closest VI file: {closest_vi_file}. Difference: {min_date_difference} days.")
        
        LST_20m = rasterio.open(fr'{lst_path}/{LST_files[v]}')
        NDVI_20m = rasterio.open(fr'{vi_path}/{closest_vi_file}')
        


In [ ]:
# Calculating the day of year (doy) and day of month (dom) to get corresponding data from ERA-5

doy = first_lst_date.timetuple().tm_yday
    
print('first_lst_date =', first_lst_date)
print('Day of the year (DOY) =', doy)

day_of_month = first_lst_date.day
print('Day of the month =', day_of_month)

month_string = first_lst_date.strftime("%B")
print('Month =', month_string)

In [ ]:
# Reading Climate data (ERA-5)

era5_dir = r'/beegfs/mirmazloumi/jupyter_playgroundpython3.10_2.0/jupyter_work/project/ERA5/2022/'
year = 2022
u_10_wind = gdal.Open(fr'{era5_dir}cropped_u10_{year}.tif')
v_10_wind = gdal.Open(fr'{era5_dir}cropped_v10_{year}.tif')
sp = gdal.Open(fr'{era5_dir}cropped_sp_{year}.tif')
ssrd = gdal.Open(fr'{era5_dir}cropped_ssrd_{year}.tif')
dt2 = gdal.Open(fr'{era5_dir}cropped_2md_{year}.tif')
ta = gdal.Open(fr'{era5_dir}cropped_2mt_{year}.tif')

In [ ]:
# Making out Nan Values from LST images

LST_data = LST_20m.read(1).astype(np.float32)
print(np.min(LST_data))
LST_data = np.ma.masked_invalid(LST_data)
LST2 = np.ma.masked_where(LST_data < 0, LST_data)
print(np.min(LST2))

In [ ]:
# Year of data
year = 2022

# DOY input
dom_input = doy
doy

In [ ]:
# Masking invalid values

NDVI_data = NDVI_20m.read(1)
NDVI2 = np.ma.masked_invalid(NDVI_data)

# LAI calculation

LAI_np = 0.57*np.exp(2.33*NDVI2)
LAI3 = np.ma.masked_where(LAI_np < 0, LAI_np)

In [ ]:
# Preparing wind speed parameter

u_date = u_10_wind.GetRasterBand(dom_input)
v_date = v_10_wind.GetRasterBand(dom_input) 
u_array = u_date.ReadAsArray()
v_array = v_date.ReadAsArray()

wind_speed = (u_array**2 + u_array**2)**0.5
wind_speed_max = np.maximum(wind_speed, 1.0)

wind_speed_dwscaled = cv2.resize(wind_speed_max, (LST2.shape[1], LST2.shape[0]), interpolation=cv2.INTER_NEAREST)

In [ ]:
# Preparing surface pressure parameter

sp_date = sp.GetRasterBand(dom_input)
sp_array = sp_date.ReadAsArray()
sp_dwscaled = cv2.resize(sp_array/100, (LST2.shape[1], LST2.shape[0]), interpolation=cv2.INTER_NEAREST)

In [ ]:
# Preparing surface solar radiation downward parameter

ssrd_date = ssrd.GetRasterBand(dom_input)
ssrd_array = ssrd_date.ReadAsArray()
ssrd_dwscaled = cv2.resize(ssrd_array/3600, (LST2.shape[1], LST2.shape[0]), interpolation=cv2.INTER_NEAREST)

In [ ]:
# Preparing dewpoint temperature

dt2_date = dt2.GetRasterBand(dom_input)
dt2_array = dt2_date.ReadAsArray()
dt2_dwscaled = cv2.resize(dt2_array, (LST2.shape[1], LST2.shape[0]), interpolation=cv2.INTER_NEAREST)

In [ ]:
# Preparing temperature

ta_date = ta.GetRasterBand(dom_input)
ta_array = ta_date.ReadAsArray()
ta_dwscaled = cv2.resize(ta_array, (LST2.shape[1], LST2.shape[0]), interpolation=cv2.INTER_NEAREST)

In [ ]:
# Defining Atmospheric emissivity and Longwave irradiance functions

def calc_emiss_atm(ea, t_a_k):
    '''Atmospheric emissivity

    Estimates the effective atmospheric emissivity for clear sky.

    Parameters
    ----------
    ea : float
        atmospheric vapour pressure (mb).
    t_a_k : float
        air temperature (Kelvin).

    Returns
    -------
    emiss_air : float
        effective atmospheric emissivity.

    References
    ----------
    .. [Brutsaert1975] Brutsaert, W. (1975) On a derivable formula for long-wave radiation
        from clear skies, Water Resour. Res., 11(5), 742-744,
        htpp://dx.doi.org/10.1029/WR011i005p00742.'''

    emiss_air = 1.24 * (ea / t_a_k)**(1. / 7.)  # Eq. 11 in [Brutsaert1975]_

    return np.asarray(emiss_air)
    
def calc_longwave_irradiance(ea, t_a_k, p, z_T, h_C):
    '''Longwave irradiance

    Estimates longwave atmospheric irradiance from clear sky.
    By default there is no lapse rate correction unless air temperature
    measurement height is considerably different than canopy height, (e.g. when
    using NWP gridded meteo data at blending height)

    Parameters
    ----------
    ea : float
        atmospheric vapour pressure (mb).
    t_a_k : float
        air temperature (K).
    p : float
        air pressure (mb)
    z_T: float
        air temperature measurement height (m), default 2 m.
    h_C: float
        canopy height (m), default 2 m,

    Returns
    -------
    L_dn : float
        Longwave atmospheric irradiance (W m-2) above the canopy
    '''

    lapse_rate = TSEB.met.calc_lapse_rate_moist(t_a_k, ea, p)
    t_a_surface = t_a_k - lapse_rate * (h_C - z_T)
    emisAtm = calc_emiss_atm(ea, t_a_surface)
    L_dn = emisAtm * TSEB.met.calc_stephan_boltzmann(t_a_surface)
    return np.asarray(L_dn)

In [ ]:
# Sun zenith angle 

sza = TSEB.met.calc_theta_s(xlat = 0.2171, xlong = 34.1089, stdlng = 40, doy = doy, year = year, ftime =10.50)

In [ ]:
# Calculating water vapor pressure (ea), pressure (p), longwave atmospheric irradiance above the canopy (L_dn), ...
#... the zero-plane displacement height, and aerodynamic roughness length

ea = TSEB.met.calc_vapor_pressure(T_K=dt2_dwscaled)
p = sp_dwscaled
L_dn = calc_longwave_irradiance(ea = ea, t_a_k = ta_dwscaled, p = sp_dwscaled, z_T = 2, h_C = 1.2)
d_0 = TSEB.res.calc_d_0(h_C=1.2)
z_0M = TSEB.res.calc_z_0M(h_C=1.2)

In [ ]:
# Fraction of difuse shortwave radiation

difvis, difnir, fvis, fnir = TSEB.rad.calc_difuse_ratio(S_dn = ssrd_dwscaled, sza = sza)

In [ ]:
# Broadband incoming beam and diffuse shortwave radiations

skyl = difvis * fvis + difnir * fnir
S_dn_dir = ssrd_dwscaled * (1.0 - skyl)
S_dn_dif = ssrd_dwscaled * skyl

In [ ]:
# Canopy and Soil spectra from https://github.com/hectornieto/pyTSEB/blob/master/Config_LocalImage.txt

emis_C=0.98 # leaf emissivity
emis_S=0.95 # soil emissivity

# Leaf spectral properties:{rho_vis_C: visible reflectance, tau_vis_C: visible transmittance, rho_nir_C: NIR reflectance, tau_nir_C: NIR transmittance}
rho_vis_C=np.full(LAI3.shape, 0.05, np.float32)
tau_vis_C=np.full(LAI3.shape, 0.08, np.float32)
rho_nir_C=np.full(LAI3.shape, 0.32, np.float32)
tau_nir_C=np.full(LAI3.shape, 0.33, np.float32) 

# Soil spectral properties:{rho_vis_S: visible reflectance, rho_nir_S: NIR reflectance}
rho_vis_S=np.full(LAI3.shape, 0.07, np.float32)
rho_nir_S=np.full(LAI3.shape, 0.25, np.float32)

In [ ]:
# Net shortwave radiation

Sn_C, Sn_S = TSEB.rad.calc_Sn_Campbell(lai = LAI3, sza = sza, S_dn_dir = S_dn_dir, S_dn_dif = S_dn_dif, fvis = fvis,
                                       fnir = fnir, rho_leaf_vis = rho_vis_C, tau_leaf_vis = tau_vis_C, rho_leaf_nir = rho_nir_C, 
                                       tau_leaf_nir = tau_nir_C, rsoilv = rho_vis_S, rsoiln = rho_nir_S, x_LAD=1, LAI_eff=None)

In [ ]:
# z_0M and d

z_0M, d = TSEB.res.calc_roughness(LAI=np.nanmean(LAI3), h_C=1.2, w_C=1, landcover=11, f_c=None)
z_0M_array = np.full(LAI3.shape, z_0M)
d_array = np.full(LAI3.shape, d)

In [ ]:
# Input Parameters to TSEB_PT
#   ----------
Tr_K = LST2
# float. Radiometric composite temperature (Kelvin).
vza = 0
# float. View Zenith Angle (degrees).
T_A_K = ta_dwscaled
# float. Air temperature (Kelvin).
u =  wind_speed_dwscaled  ## ***ERA5 two para
# float. Wind speed above the canopy (m s-1).
ea = ea
# float. Water vapour pressure above the canopy (mb).
p = sp_dwscaled 
# float. Atmospheric pressure (mb), use 1013 mb by default.
Sn_C = Sn_C
# float. Canopy net shortwave radiation (W m-2).
Sn_S =  Sn_S
# float. Soil net shortwave radiation (W m-2).
L_dn = L_dn
# float. Downwelling longwave radiation (W m-2).
LAI = LAI3
# float. Effective Leaf Area Index (m2 m-2).
h_C = 1.2 
# float. Canopy height (m).
emis_C = 0.98
# float. Leaf emissivity.
emis_S = 0.95
# float. Soil emissivity.
z_0M = z_0M_array 
# float. Aerodynamic surface roughness length for momentum transfer (m).
d_0 = d_array 
# float. Zero-plane displacement height (m).
z_u = 10 
# float. Height of measurement of windspeed (m).
z_T = 2 
# float. Height of measurement of air temperature (m).
# leaf_width : 
# float, optional average/effective leaf width (m).
z0_soil = 0.01    #optional
# float, bare soil aerodynamic roughness length (m).
alpha_PT = 1.26  #optional
#  float, optional. Priestley Taylor coeffient for canopy potential transpiration, use 1.26 by default.
x_LAD = 1        #optional
# float. Campbell 1990 leaf inclination distribution function chi parameter.
f_c = 1          #optional
#float. Fractional cover.
f_g = 1        # optional
# float. Fraction of vegetation that is green.
w_C = 1       #optional
# float. Canopy width to height ratio.

In [ ]:
# ET estimation

output = TSEB.TSEB_PT(Tr_K, vza, T_A_K, u, ea, p, Sn_C, Sn_S, L_dn, LAI3, h_C, emis_C, emis_S, 
                      z_0M, d_0, z_u, z_T, resistance_form=None, calcG_params=None, const_L=None, 
                      kB=0.0, massman_profile=None, verbose=True)

In [ ]:
et_daily = TSEB.met.flux_2_evaporation(output[6], t_k=T_A_K, time_domain=24)

In [ ]:
# Exporting actual ET as GeoTiff files        
        
output_file = f'/beegfs/mirmazloumi/jupyter_playgroundpython3.10_2.0/jupyter_work/project/ET_results/ET_2022_G10_Monthly/actual_ET_G10_20m_monthly_{first_lst_date_string}.tif'
# Busia: r'H:\ADM-Kenya\Results\ET\Busia_June2021\ET_Busia_20m_03062021_mm.tif'
# arid_aoi: 
# Get the metadata from the NDVI raster
LST_metadata = LST_20m.meta

et_daily_p = np.ma.masked_where(et_daily < 0, et_daily)
et_daily_p = et_daily_p.filled(np.nan)  # Fill masked values with np.nan

# Create the destination metadata
dst_metadata = {
    'driver': 'GTiff',
    'dtype': et_daily_p.dtype,  # Use the dtype of the masked array
    'nodata': np.nan,  # Set nodata to np.nan
    'width': et_daily_p.shape[1],
    'height': et_daily_p.shape[0],
    'count': 1,  # Assuming a single-band GeoTIFF
    'crs': LST_metadata['crs'],
    'transform': LST_metadata['transform']
}

# Save the array as a GeoTIFF
with rasterio.open(output_file, 'w', **dst_metadata) as dst:
    dst.write(et_daily_p, 1)